In [1]:
import os
import sys

from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np
import SimpleITK as sitk
import nrrd
import vtk

import torch
from torch import nn
from torch.nn import functional as F
from torchvision import transforms

import pytorch_lightning as pl
import pickle
import monai 
import glob 
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

sys.path.append('/mnt/raid/C1_ML_Analysis/source/famli-ultra-sim/')
sys.path.append('/mnt/raid/C1_ML_Analysis/source/famli-ultra-sim/dl')
import dl.transforms.ultrasound_transforms as ultrasound_transforms
import dl.nets.us_simulation_jit as us_simulation_jit
import dl.nets.us_simu as us_simu

import importlib

from dl.nets.layers import TimeDistributed

sys.path.append('/mnt/raid/C1_ML_Analysis/source/ShapeAXI/src')
from shapeaxi import utils as saxi_utils


In [2]:
mount_point = '/mnt/raid/C1_ML_Analysis/simulated_data_export/animation_export/'

importlib.reload(us_simu)
vs = us_simu.VolumeSamplingBlindSweep(mount_point=mount_point, simulation_fov_fn='simulation_fov.stl', simulation_ultrasound_plane_fn='ultrasound_grid.stl')
vs.init_probe_params_from_pos(os.path.join(mount_point, 'all_poses/frame_0001/probe_paths'))
vs = vs.cuda()

In [3]:

# diffusor = sitk.ReadImage('/mnt/famli_netapp_shared/C1_ML_Analysis/src/blender/simulated_data_export/studies_merged/FAM-025-0447-5.nrrd')
# diffusor_np = sitk.GetArrayFromImage(diffusor)
# diffusor_t = torch.tensor(diffusor_np.astype(int))

# diffusor_spacing = torch.tensor(diffusor.GetSpacing()).flip(dims=[0])
# diffusor_size = torch.tensor(diffusor.GetSize()).flip(dims=[0])

# diffusor_origin = torch.tensor(diffusor.GetOrigin()).flip(dims=[0])
# diffusor_end = diffusor_origin + diffusor_spacing * diffusor_size
# print(diffusor_size)
# print(diffusor_spacing)
# print(diffusor_t.shape)
# print(diffusor_origin)
# print(diffusor_end)



# diffusor_np, diffusor_head = nrrd.read(os.path.join(mount_point, 'breech/frame_0001.nrrd'))
# diffusor_t = torch.tensor(diffusor_np.astype(int)).permute(2, 1, 0)
# print(diffusor_head)
# diffusor_size = torch.tensor(diffusor_head['sizes'])
# diffusor_spacing = torch.tensor(np.diag(diffusor_head['space directions']))

# diffusor_origin = torch.tensor(diffusor_head['space origin']).flip(dims=[0])
# diffusor_end = diffusor_origin + diffusor_spacing * diffusor_size



diffusor = sitk.ReadImage(os.path.join(mount_point, 'all_poses/frame_0001.nrrd'))
diffusor_t = torch.tensor(sitk.GetArrayFromImage(diffusor).astype(int))  # Convert to (z, y, x) format
diffusor_size = torch.tensor(diffusor.GetSize())
diffusor_spacing = torch.tensor(diffusor.GetSpacing())
diffusor_origin = torch.tensor(diffusor.GetOrigin())
diffusor_end = diffusor_origin + diffusor_spacing * diffusor_size

print(diffusor.GetDirection())
print(diffusor_spacing)
print(diffusor_t.shape)
print(diffusor_size)
print(diffusor_origin)
print(diffusor_end)



In [4]:
# fig = px.imshow(diffusor_t.flip(dims=[1]).squeeze().cpu().numpy(), animation_frame=0, binary_string=True)
# fig.show()

In [5]:
diffusor_batch_t = diffusor_t.cuda().float().unsqueeze(0).unsqueeze(0)


diffusor_origin_batch = diffusor_origin[None, :].cuda()
diffusor_end_batch = diffusor_end[None, :].cuda()

diffusor_spacing_batch_t = diffusor_spacing.unsqueeze(0).cuda()

print(diffusor_batch_t.shape, diffusor_origin_batch.shape, diffusor_origin_batch.shape)
# print(diffusor_origin_batch.shape)

# diffusor_in_fov_t = vs.diffusor_in_fov(diffusor_batch_t, diffusor_origin_batch.cuda(), diffusor_end_batch.cuda())


In [6]:

reader = vtk.vtkSTLReader()
reader.SetFileName(os.path.join(mount_point, 'all_poses/frame_0001/fetus/Fetus_Model.stl'))
reader.Update()
surf = reader.GetOutput()
V, F = saxi_utils.PolyDataToTensors_v_f(surf)

In [7]:
df = pd.read_csv(os.path.join(mount_point, "shapes_intensity_map_nrrd.csv"))
sk_tags = torch.tensor(list(df.query("group=='skeleton' or group=='skull' or group=='arms' or group=='legs'")['label'])).cuda()
# sk_tags = torch.tensor(list(df.query("group=='lady'")['label'])).cuda()

In [8]:
mask = torch.isin(diffusor_batch_t.squeeze(1), sk_tags)
coords = []
for bidx, m in enumerate(mask):
    idx = torch.argwhere(m)    
    xyz = idx*diffusor_spacing_batch_t[bidx] + diffusor_origin_batch[bidx]
    coords.append(xyz)



In [9]:
simulation_planes = vs.transform_simulation_ultrasound_plane_tag("R0")
simulation_planes.shape

In [10]:
idx = torch.randint(0, len(coords[0]), (10000,))
VDiff = coords[0][idx]
SN = 0


trace1 = go.Scatter3d(x=V[:,0].detach().cpu().numpy(), y=V[:,1].detach().cpu().numpy(), z=V[:,2].detach().cpu().numpy(), mode='markers', marker=dict(
    size=2,
    color=V[:,2].detach().cpu().numpy(),                # set color to an array/list of desired values
    colorscale='jet',   # choose a colorscale
    # opacity=0.5
))



trace2 = go.Scatter3d(x=VDiff[:,0].detach().cpu().numpy(), y=VDiff[:,1].detach().cpu().numpy(), z=VDiff[:,2].detach().cpu().numpy(), mode='markers', marker=dict(
    size=2,
    color=VDiff[:,2].detach().cpu().numpy(),                # set color to an array/list of desired values
    colorscale='viridis',   # choose a colorscale
    # opacity=0.5
))

VSimu = simulation_planes[100:110].reshape(-1, 3)
VSimu = VSimu[torch.randint(0, len(VSimu), (10000,))]

SN = 0
trace3 = go.Scatter3d(x=VSimu[:,0].detach().cpu().numpy(), y=VSimu[:,1].detach().cpu().numpy(), z=VSimu[:,2].detach().cpu().numpy(), mode='markers', marker=dict(
    size=2,
    color=VSimu[:,2].detach().cpu().numpy(),                # set color to an array/list of desired values
    colorscale='turbo',   # choose a colorscale
    # opacity=0.5
))

VPos = vs.probe_origins_R0
trace4 = go.Scatter3d(x=VPos[:,0].detach().cpu().numpy(), y=VPos[:,1].detach().cpu().numpy(), z=VPos[:,2].detach().cpu().numpy(), mode='markers', marker=dict(
    size=8,
    color='magenta',
    # opacity=0.5
))

# Combine both traces in the same figure
fig = go.Figure(data=[trace1, trace2, trace3, trace4])
fig.show()

In [11]:
# fig = px.imshow(diffusor_in_fov_t[0].squeeze().flip(dims=[1]).cpu().numpy(), animation_frame=0, binary_string=True)
# fig.show()
# fig = px.imshow(diffusor_in_fov_t[1].squeeze().flip(dims=[1]).cpu().numpy(), animation_frame=0, binary_string=True)
# fig.show()

In [12]:
importlib.reload(us_simulation_jit)
us_simulator_cut = us_simulation_jit.MergedLinearLabel11()
us_simulator_cut = us_simulator_cut.cuda()
grid, inverse_grid, mask_fan = us_simulator_cut.init_grids(256, 256, 128.0, -30.0, 20.0, 215.0, 0.7853981633974483)
us_simulator_cut_td = TimeDistributed(us_simulator_cut, time_dim=2).eval().cuda()


In [20]:


#tags = ['M', 'L0', 'L1', 'R0', 'R1', 'C1', 'C2', 'C3', 'C4']
t_idx = 7

tag = vs.tags[t_idx]


# sampled_sweep = vs.diffusor_sampling_tag(tag, diffusor_batch_t.to(torch.float).cuda(), diffusor_origin_batch.to(torch.float).cuda(), diffusor_end_batch.to(torch.float).cuda())
# sampled_sweep_simu = torch.cat([us_simulator_cut_td(ss.unsqueeze(dim=0), grid.cuda(), inverse_grid.cuda(), mask_fan.cuda()) for ss in sampled_sweep], dim=0)


sampled_sweep_simu = vs.get_sweep(diffusor_batch_t.to(torch.float).cuda(), diffusor_origin_batch.to(torch.float).cuda(), diffusor_end_batch.to(torch.float).cuda(), tag, use_random=False, simulator=None, grid=grid.cuda(), inverse_grid=inverse_grid.cuda(), mask_fan=mask_fan.cuda())
print(sampled_sweep_simu.shape)


In [21]:
fig = px.imshow(sampled_sweep_simu.squeeze().detach().cpu().numpy(), animation_frame=0, binary_string=True)
fig.show()

In [15]:
reader = vtk.vtkSTLReader()
reader.SetFileName(os.path.join(mount_point, 'ultrasound_grid.stl'))
reader.Update()
simulation_ultrasound_plane = reader.GetOutput()
simulation_ultrasound_plane_V, simulation_ultrasound_plane_F = saxi_utils.PolyDataToTensors_v_f(simulation_ultrasound_plane)
print(simulation_ultrasound_plane.GetBounds())

In [16]:
simulation_ultrasound_plane_V

In [17]:
vs.simulation_ultrasound_plane_mesh_grid